In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext sql

In [10]:
%sql sqlite:///results.db

### Creating Indexes & Adding Summary Columns

Before we dive into all kinds of data analysis, let's create some table indexes to speed things up, and some additional columns that will come in handy.

Note: The dataset on Zenodo already includes these things.

In [ ]:
%%sql 
create index alias_name_idx on alias (name);
create index alias_value_idx on alias (value);
create index command_name_idx on command (name);
create index command_arguments_idx on command (arguments);
create index command_alias_id_idx on command (alias_id);
create index argument_name_idx on argument (name);
create index argument_command_id_idx on argument (command_id);

In [ ]:
%%sql
begin transaction;

create temp table alias_num_commands as 
select alias.alias_id as alias_id, count(command.alias_id) as num_commands
from alias join command using (alias_id)
group by alias.alias_id;
create index alias_num_commands_idx on alias_num_commands (alias_id);

alter table alias add column num_commands integer not null default 0;
update alias set num_commands = (
    select num_commands 
    from alias_num_commands 
    where alias_num_commands.alias_id = alias.alias_id
);
create index alias_num_commands_idx on alias (num_commands);

drop table alias_num_commands;

commit;

In [ ]:
%%sql
begin transaction;

create temp table command_num_arguments as
select command.command_id as command_id, count(argument.command_id) as num_arguments
from command join argument using (command_id)
group by command.command_id;

insert into command_num_arguments
select command.command_id, 0
from command
where command.command_id not in (select command_id from command_num_arguments);

create index command_num_arguments_idx on command_num_arguments (command_id);

alter table command add column num_arguments integer not null default 0;
update command set num_arguments = (
    select num_arguments
    from command_num_arguments
    where command_num_arguments.command_id = command.command_id
);
create index command_num_arguments_idx on command (num_arguments);

drop table command_num_arguments;

commit;

# Summary Statistics

In [35]:
%sql select count(*) as number_of_aliases from alias

 * sqlite:///results.db
Done.


number_of_aliases
2204199


In [36]:
%sql select count(*) as number_of_commands from command

 * sqlite:///results.db
Done.


number_of_commands
2534167


In [37]:
%sql select count(*) as number_of_arguments from argument

 * sqlite:///results.db
Done.


number_of_arguments
3630423


# Top alias definitions

In [38]:
%%sql top_aliases <<
with total as (select count(*) as total from alias) 
select
    '\verb|' || name || '|' as 'Name',
    '=',
    '\verb|' || value || '|' as 'Value', 
    '\num{{' || count(*) || '}}' as '\#', 
    '\num{{' || round(count(*)*100.0/total,2) || '}}' as '\%' 
from alias
join total 
group by name, value 
order by count(*) desc 
limit 10

 * sqlite:///results.db
Done.
Returning data to local variable top_aliases


In [39]:
print(top_aliases.DataFrame().to_latex(index=False, escape=False, column_format='rclrr'))

\begin{tabular}{rclrr}
\toprule
         Name & '=' &                                              Value &           \# &          \% \\
\midrule
    \verb|ls| &   = &                             \verb|ls --color=auto| &  \num{37224} &  \num{1.69} \\
  \verb|grep| &   = &                           \verb|grep --color=auto| &  \num{33192} &  \num{1.51} \\
 \verb|egrep| &   = &                          \verb|egrep --color=auto| &  \num{25488} &  \num{1.16} \\
 \verb|fgrep| &   = &                          \verb|fgrep --color=auto| &  \num{25114} &  \num{1.14} \\
    \verb|la| &   = &                                       \verb|ls -A| &  \num{21912} &  \num{0.99} \\
     \verb|l| &   = &                                      \verb|ls -CF| &  \num{20037} &  \num{0.91} \\
    \verb|ll| &   = &                                     \verb|ls -alF| &  \num{17192} &  \num{0.78} \\
 \verb|alert| &   = &  \verb|notify-send --urgency=low -i "$([ $? = 0... &  \num{15635} &  \num{0.71} \\
    \verb|..| 

# Top alias names, commands, and arguments

In [40]:
%%sql top_names <<
with total as (select count(*) as total from alias) 
select 
    '\verb|' || name || '|' as 'Alias Name', 
    '\num{{' || count(*) || '}}' as '\#', 
    '\num{{' || round(count(*)*100.0/total,2) || '}}' as '\%' 
from alias 
join total 
group by name 
order by count(*) desc 
limit 15

 * sqlite:///results.db
Done.
Returning data to local variable top_names


In [41]:
print(top_names.DataFrame().to_latex(index=False, escape=False, column_format='lrr'))

\begin{tabular}{lrr}
\toprule
   Alias Name &           \# &          \% \\
\midrule
    \verb|ls| &  \num{83782} &   \num{3.8} \\
    \verb|ll| &  \num{62465} &  \num{2.83} \\
  \verb|grep| &  \num{44479} &  \num{2.02} \\
    \verb|la| &  \num{43760} &  \num{1.99} \\
     \verb|l| &  \num{39539} &  \num{1.79} \\
 \verb|egrep| &  \num{27852} &  \num{1.26} \\
 \verb|fgrep| &  \num{27067} &  \num{1.23} \\
    \verb|..| &  \num{17145} &  \num{0.78} \\
 \verb|alert| &  \num{15999} &  \num{0.73} \\
    \verb|rm| &  \num{15715} &  \num{0.71} \\
    \verb|vi| &  \num{15521} &   \num{0.7} \\
    \verb|cp| &  \num{15184} &  \num{0.69} \\
    \verb|mv| &  \num{13447} &  \num{0.61} \\
    \verb|gs| &  \num{13391} &  \num{0.61} \\
   \verb|vim| &  \num{13165} &   \num{0.6} \\
\bottomrule
\end{tabular}



In [42]:
%%sql top_commands <<
with total as (select count(*) as total from command) 
select 
    '\verb|' || name || '|' as 'Command', 
    '\num{{' || count(*) || '}}' as '\#', 
    '\num{{' || round(count(*)*100.0/total,2) || '}}' as '\%' 
from command 
join total 
group by name 
order by count(*) desc 
limit 15

 * sqlite:///results.db
Done.
Returning data to local variable top_commands


In [43]:
print(top_commands.DataFrame().to_latex(index=False, escape=False, column_format='lrr'))

\begin{tabular}{lrr}
\toprule
       Command &            \# &           \% \\
\midrule
    \verb|git| &  \num{327786} &  \num{12.93} \\
     \verb|ls| &  \num{260156} &  \num{10.27} \\
     \verb|cd| &  \num{166632} &   \num{6.58} \\
   \verb|grep| &   \num{89598} &   \num{3.54} \\
    \verb|vim| &   \num{46545} &   \num{1.84} \\
   \verb|echo| &   \num{31735} &   \num{1.25} \\
    \verb|ssh| &   \num{31352} &   \num{1.24} \\
  \verb|egrep| &   \num{30953} &   \num{1.22} \\
 \verb|docker| &   \num{27946} &    \num{1.1} \\
  \verb|fgrep| &   \num{27054} &   \num{1.07} \\
     \verb|rm| &   \num{26679} &   \num{1.05} \\
 \verb|source| &   \num{22189} &   \num{0.88} \\
   \verb|tmux| &   \num{21788} &   \num{0.86} \\
  \verb|clear| &   \num{19567} &   \num{0.77} \\
     \verb|cp| &   \num{17754} &    \num{0.7} \\
\bottomrule
\end{tabular}



In [44]:
%%sql top_arguments <<
with total as (select count(*) as total from argument) 
select 
    '\verb|' || name || '|' as 'Argument', 
    '\num{{' || count(*) || '}}' as '\#', 
    '\num{{' || round(count(*)*100.0/total,2) || '}}' as '\%' 
from argument 
join total 
group by name 
order by count(*) desc 
limit 20

 * sqlite:///results.db
Done.
Returning data to local variable top_arguments


In [45]:
print(top_arguments.DataFrame().to_latex(index=False, escape=False, column_format='lrr'))

\begin{tabular}{lrr}
\toprule
            Argument &            \# &          \% \\
\midrule
 \verb|--color=auto| &  \num{153931} &  \num{4.24} \\
           \verb|-i| &   \num{70640} &  \num{1.95} \\
           \verb|-a| &   \num{42910} &  \num{1.18} \\
           \verb|-l| &   \num{39519} &  \num{1.09} \\
           \verb|-v| &   \num{35295} &  \num{0.97} \\
       \verb|commit| &   \num{32238} &  \num{0.89} \\
           \verb|-A| &   \num{31883} &  \num{0.88} \\
       \verb|status| &   \num{28348} &  \num{0.78} \\
           \verb|-f| &   \num{27913} &  \num{0.77} \\
         \verb|push| &   \num{27423} &  \num{0.76} \\
            \verb|.| &   \num{26627} &  \num{0.73} \\
          \verb|add| &   \num{26364} &  \num{0.73} \\
          \verb|log| &   \num{24442} &  \num{0.67} \\
           \verb|-d| &   \num{24364} &  \num{0.67} \\
           \verb|..| &   \num{24145} &  \num{0.67} \\
           \verb|-p| &   \num{23882} &  \num{0.66} \\
           \verb|-m| &   \num{23538} &  \nu

# Top commands with their top arguments and top aliases

In [48]:
%%sql top3 <<
with 
    top_commands as (
        select name as cmd_name, count(*) as cmd_count 
        from command 
        where cmd_name in ('git', 'ls')
        group by cmd_name 
        order by cmd_count desc 
    ), 
    top_arguments as (
        select name as cmd_name, arguments as cmd_args, count(*) as arg_count 
        from command 
        where cmd_name in (select cmd_name from top_commands) 
        group by cmd_name, cmd_args 
        order by cmd_name, arg_count desc
    ),
    X as (
        select *, row_number() over (partition by cmd_name order by arg_count desc) as num
        from top_arguments
    ),
    top_aliases as (
        select 
            command.name as cmd_name, 
            command.arguments as cmd_args,
            alias.name as alias_name,
            count(*) as alias_count
        from alias join command using (alias_id)
        where cmd_name in (select cmd_name from top_commands)
        group by cmd_name, cmd_args, alias_name 
        order by cmd_name, cmd_args, alias_count desc
    ),
    Y as (
        select *, row_number() over (partition by cmd_name, cmd_args order by alias_count desc) as num
        from top_aliases
    )

select
    cmd_count,
    cmd_name,
    round(arg_count*100.0/cmd_count, 2) as args_per,
    cmd_args,
    alias_name, 
    round(alias_count*100.0/arg_count, 2) as alias_per

from top_commands join X using (cmd_name) join Y using (cmd_name, cmd_args)
where X.num <= 10 and Y.num <= 3
and alias_per > 5
order by cmd_count desc, arg_count desc, alias_count desc;

 * sqlite:///results.db
Done.
Returning data to local variable top3


In [49]:
def format_alias(x):
    return '\\verb|%s| (\\num{%.2f})' % (x['alias_name'], x['alias_per'])
          
df = top3.DataFrame()
df['alias_fmt'] = df.apply(format_alias, axis=1)
df = df.groupby(['cmd_count','cmd_name','args_per','cmd_args']).agg(aliases=('alias_fmt', ', '.join)).reset_index()
df.sort_values(by=['cmd_count','cmd_name','args_per'],ascending=[False,True,False],inplace=True)

df.drop(['cmd_count'], axis=1, inplace=True)
df['cmd_name'] = df.apply(lambda x: '\\verb|%s|' % x['cmd_name'], axis=1)
df['args_per'] = df.apply(lambda x: '\\num{%.2f}' % x['args_per'], axis=1)
df['cmd_args'] = df.apply(lambda x: '\\verb|%s|' % x['cmd_args'], axis=1)

with pd.option_context("max_colwidth", 1000):
    print(df.to_latex(escape=False,index=False,column_format='lrll',header=['Command','\%','Arguments','Aliases (\%)']))

\begin{tabular}{lrll}
\toprule
    Command &           \% &            Arguments &                                                                Aliases (\%) \\
\midrule
 \verb|git| &   \num{5.85} &        \verb|status| &                           \verb|gs| (\num{54.27}), \verb|gst| (\num{19.19}) \\
 \verb|git| &   \num{3.48} &              \verb|| &                             \verb|g| (\num{75.71}), \verb|gti| (\num{5.74}) \\
 \verb|git| &   \num{3.20} &      \verb|checkout| &  \verb|gco| (\num{50.52}), \verb|gc| (\num{13.87}), \verb|gch| (\num{7.56}) \\
 \verb|git| &   \num{3.18} &          \verb|push| &  \verb|gp| (\num{46.73}), \verb|gps| (\num{9.23}), \verb|push| (\num{7.56}) \\
 \verb|git| &   \num{3.16} &          \verb|diff| &                                                     \verb|gd| (\num{79.89}) \\
 \verb|git| &   \num{2.86} &          \verb|pull| &  \verb|gpl| (\num{18.30}), \verb|gl| (\num{16.59}), \verb|gp| (\num{15.07}) \\
 \verb|git| &   \num{2.78} &        \verb|b